### This notebook is meant to help determine how to setup a Markov Chain transitin matrix to reflect a desired MTTF

In [2]:
import sys
sys.path.append('..')

In [ ]:
from shipClass.MarkovChain import MarkovChain
import numpy as np


states = {0: 'working', 
          1: 'failed'}

MTTF = 10
transition_mat = np.array([[1/MTTF, 1-1/MTTF],
                           [0, 1]])         

# simulate n samples over p time steps and determine the simulated MTTF
simulation_duration = 100  # p
num_test_chains = 1000    # n
failure_times = np.zeros(num_test_chains)
for i in range(num_test_chains):
    testMarkovChain = MarkovChain(states, transition_mat) 
    testMarkovChain.simulate(simulation_duration)
    failure_times[i] = testMarkovChain.failure_time
    
    
simulated_MTTF = np.mean(failure_times)
max_simulated_MTTF = np.max(failure_times)
min_simulated_MTTF = np.min(failure_times)
print('Simulated MTTF: ', simulated_MTTF)
print('Max Simulated MTTF: ', max_simulated_MTTF)
print('Min Simulated MTTF: ', min_simulated_MTTF)

Simulated MTTF:  1.114
Max Simulated MTTF:  3.0
Min Simulated MTTF:  1.0
